In [13]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/Users/marclambertes/Python/Matches/Men/2024-2025/Bundesliga 2024-2025/St. Pauli 0-2 Dortmund.csv')

# Identify columns that contain '/qualifierId'
type_cols = [col for col in df.columns if '/qualifierId' in col]

# Initialize 'endX' and 'endY' columns with 0.0
df['endX'] = 0.0
df['endY'] = 0.0

# Iterate through each row to find and set 'endX' and 'endY' values
for i in range(len(df)):
    df1 = df.iloc[i:i+1, :]
    for j in range(len(type_cols)):
        col = df1[type_cols[j]].values[0]
        if col == 140:
            endx = df1.loc[:, 'qualifier/%i/value' % j].values[0]
            df.at[i, 'endX'] = endx
        elif col == 141:
            endy = df1.loc[:, 'qualifier/%i/value' % j].values[0]
            df.at[i, 'endY'] = endy

# Ensure events are sorted in chronological order if there's a timestamp column
if 'timeStamp' in df.columns:
    df = df.sort_values(by='timeStamp')

# Convert timestamp to datetime format (remove timezone)
if df['timeStamp'].dtype != 'datetime64[ns]':
    df['timeStamp'] = pd.to_datetime(df['timeStamp'], utc=True).dt.tz_localize(None)  # Removes timezone

# Initialize sequence tracking
sequences = []
current_sequence = []
current_contestant = None
sequence_id = 1  # Unique identifier for each sequence

# Iterate over the DataFrame rows
for _, row in df.iterrows():
    if current_contestant is None or row['contestantId'] == current_contestant:
        # Append action to current sequence
        current_sequence.append(row)
    else:
        # New contestantId → Store the previous sequence if it ends with a shot
        if current_sequence:
            last_action = current_sequence[-1]
            if last_action['typeId'] in [13, 14, 15, 16]:  # Check if last action was a shot
                seq_df = pd.DataFrame(current_sequence)

                # Calculate event duration (time difference between events)
                seq_df['event_duration'] = seq_df['timeStamp'].diff().fillna(pd.Timedelta(seconds=0))

                # Add sequenceId column
                seq_df['sequenceId'] = sequence_id
                sequence_id += 1

                sequences.append(seq_df)
        
        current_sequence = [row]  # Start new sequence
    
    # Update current contestant
    current_contestant = row['contestantId']

# Append last sequence if it exists and ends with a shot
if current_sequence:
    last_action = current_sequence[-1]
    if last_action['typeId'] in [13, 14, 15, 16]:
        seq_df = pd.DataFrame(current_sequence)

        # Calculate event duration
        seq_df['event_duration'] = seq_df['timeStamp'].diff().fillna(pd.Timedelta(seconds=0))

        # Add sequenceId column
        seq_df['sequenceId'] = sequence_id

        sequences.append(seq_df)

# Combine all sequences into a single DataFrame
if sequences:
    final_df = pd.concat(sequences, ignore_index=True)

    # Save to Excel (single sheet)
    output_file = "/Users/marclambertes/Python/Shot_Sequences.xlsx"
    final_df[['sequenceId', 'contestantId', 'playerName', 'typeId', 'timeStamp', 'event_duration', 'x', 'y', 'endX', 'endY']].to_excel(output_file, sheet_name="All Sequences", index=False)

    print(f"Sequences ending with a shot saved to {output_file}")
else:
    print("No sequences ending with a shot found.")


Sequences ending with a shot saved to /Users/marclambertes/Python/Shot_Sequences.xlsx


In [1]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/Users/marclambertes/Python/Matches/Men/2024-2025/Bundesliga 2024-2025/St. Pauli 0-2 Dortmund.csv')

# Identify columns that contain '/qualifierId'
type_cols = [col for col in df.columns if '/qualifierId' in col]

# Initialize 'endX' and 'endY' columns with 0.0
df['endX'] = 0.0
df['endY'] = 0.0

# Iterate through each row to find and set 'endX' and 'endY' values
for i in range(len(df)):
    df1 = df.iloc[i:i+1, :]
    for j in range(len(type_cols)):
        col = df1[type_cols[j]].values[0]
        if col == 140:
            endx = df1.loc[:, 'qualifier/%i/value' % j].values[0]
            df.at[i, 'endX'] = endx
        elif col == 141:
            endy = df1.loc[:, 'qualifier/%i/value' % j].values[0]
            df.at[i, 'endY'] = endy

# Ensure events are sorted in chronological order if there's a timestamp column
if 'timeStamp' in df.columns:
    df = df.sort_values(by='timeStamp')

# Convert timestamp to datetime format (remove timezone)
if df['timeStamp'].dtype != 'datetime64[ns]':
    df['timeStamp'] = pd.to_datetime(df['timeStamp'], utc=True).dt.tz_localize(None)  # Removes timezone

# Initialize sequence tracking
sequences = []
current_sequence = []
current_contestant = None
sequence_id = 1  # Unique identifier for each sequence

# Iterate over the DataFrame rows
for _, row in df.iterrows():
    if current_contestant is None or row['contestantId'] == current_contestant:
        # Append action to current sequence
        current_sequence.append(row)
    else:
        # New contestantId → Store the previous sequence if it ends with a shot
        if current_sequence:
            last_action = current_sequence[-1]
            if last_action['typeId'] in [13, 14, 15, 16]:  # Check if last action was a shot
                seq_df = pd.DataFrame(current_sequence)

                # Calculate event duration (time difference between events)
                seq_df['event_duration'] = seq_df['timeStamp'].diff().fillna(pd.Timedelta(seconds=0))

                # Add sequenceId column
                seq_df['sequenceId'] = sequence_id
                sequence_id += 1

                sequences.append(seq_df)
        
        current_sequence = [row]  # Start new sequence
    
    # Update current contestant
    current_contestant = row['contestantId']

# Append last sequence if it exists and ends with a shot
if current_sequence:
    last_action = current_sequence[-1]
    if last_action['typeId'] in [13, 14, 15, 16]:
        seq_df = pd.DataFrame(current_sequence)

        # Calculate event duration
        seq_df['event_duration'] = seq_df['timeStamp'].diff().fillna(pd.Timedelta(seconds=0))

        # Add sequenceId column
        seq_df['sequenceId'] = sequence_id

        sequences.append(seq_df)

# Combine all sequences into a single DataFrame
if sequences:
    final_df = pd.concat(sequences, ignore_index=True)

    # Calculate the contribution of each action in the sequence
    final_df['contribution'] = 0.0

    for seq_id in final_df['sequenceId'].unique():
        seq = final_df[final_df['sequenceId'] == seq_id]
        num_actions = len(seq)
        for i, (index, row) in enumerate(seq.iterrows()):
            # Assign higher weight to actions closer to the shot
            final_df.at[index, 'contribution'] = (i + 1) / num_actions

    # Save to Excel (single sheet)
    output_file = "/Users/marclambertes/Python/Shot_Sequences_with_Contribution.xlsx"
    final_df[['sequenceId', 'contestantId', 'playerName', 'typeId', 'timeStamp', 'event_duration', 'x', 'y', 'endX', 'endY', 'contribution']].to_excel(output_file, sheet_name="All Sequences", index=False)

    print(f"Sequences ending with a shot saved to {output_file}")
else:
    print("No sequences ending with a shot found.")

Sequences ending with a shot saved to /Users/marclambertes/Python/Shot_Sequences_with_Contribution.xlsx
